# Multi-label image classification with ImageDataGenerator

In [3]:
# sklearn
from sklearn.model_selection import train_test_split
import numpy as np

# tensorflow and keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Input, Conv2D, Flatten, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Accuracy, CategoricalAccuracy
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.utils import to_categorical
import datetime
import pandas as pd

In [4]:

MAIN_PATH = "../raw_data/wikiart/" # Path to the directory which contains CSVs and the folder 'dataset'
IMAGES = "dataset"
CSV_NAME = "wikiart-movement-genre_True-class_3-merge_test1-n_100_max.csv" # Nwikiart-movement-genre_True-class_3-merge_test1-n_1000_max.csvme of the CSV we want to use
NUM_MOVEMENT = 3 # Number of movements to classify
NUM_GENRE = 10 # Number of genres to classify
IMG_HEIGHT = IMG_WIDTH = 224 # Model's inputs shapes

USER = "gregoire" # Choose between 'common', 'pablo', 'quentin', 'gregoire', 'alex'
MODEL_NAME = "Custom v1" # Set the name of the model 

'''----------------------------------
Load the CSV
----------------------------------'''
BATCH_SIZE = 32
EPOCHS = 50


'''----------------------------------
Load the CSV
----------------------------------'''
raw_df = pd.read_csv(MAIN_PATH + CSV_NAME)
df = raw_df.iloc[:, 0:3]
assert type(df) == type(pd.DataFrame()) # Check if we created a dataframe
assert df.iloc[:, 1].nunique() ==  NUM_MOVEMENT # Check if we have the correct number of movements
assert df.iloc[:, 2].nunique() ==  NUM_GENRE # Check if we have the correct number of genres

print(f"Number of images : {df.shape[0]}")

'''----------------------------------
OHE
----------------------------------'''

df_genre_ohe = pd.get_dummies(df['genre'])
df_mov_ohe = pd.get_dummies(df['movement'])

df = pd.concat([df,df_genre_ohe, df_mov_ohe], axis=1)


Number of images : 300


In [14]:
'''----------------------------------
Train, test, val split
----------------------------------'''
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True)
df_train, df_val = train_test_split(df_train, test_size=0.2, shuffle=True)
# assert type(df_train) == type(pd.DataFrame()) # Check if we created dataframes
# assert type(df_test) == type(pd.DataFrame())
# assert type(df_val) == type(pd.DataFrame())

print(f"Number of images in train set : {df_train.shape[0]}")
print(f"Number of images in val set : {df_val.shape[0]}")
print(f"Number of images in test set : {df_test.shape[0]}")
print('\n')


'''----------------------------------
Setup outputs columns
----------------------------------'''
#assert len(list(df.columns[1:])) == 2 # Check if we have two outputs columns
#columns=list(df.columns[1:])

columns_genre = list(df_genre_ohe.columns)
columns_mov = list(df_mov_ohe.columns)
columns = columns_mov + columns_genre 

'''----------------------------------
Train ImageDataGenerator
----------------------------------'''
train_datagen = ImageDataGenerator( # This generator is only used to train data because it has data augmentation and we do not want to augment data from the test or val set
    rescale=1./255,
    rotation_range=15,
    zoom_range=(0.95, 0.95),
    horizontal_flip=True,
    dtype=tf.float32
    )

#assert type(train_datagen) == type(ImageDataGenerator())

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train, # Dataset used to get the path (column filename) and the linked outputs
    directory=MAIN_PATH + IMAGES, # Path to the images
    x_col="file_name", # Column with the name of the images that the generator will get from the directory
    y_col=columns, # Columns with the output of the images that the generator will get from the csv
    batch_size=BATCH_SIZE,
    seed=None,
    shuffle=True,
    class_mode="raw", # numpy array of values in y_col columns
    target_size=(IMG_HEIGHT, IMG_WIDTH), # Resize the images to the input shape of the model
    data_format='channels_last'
    ) 


'''----------------------------------
Test and Val ImageDataGenerator
----------------------------------'''
test_val_datagen = ImageDataGenerator(  # We use a new generator without data augmentation
    rescale=1./255
    )

val_generator = test_val_datagen.flow_from_dataframe(
    dataframe=df_val, 
    directory=MAIN_PATH + IMAGES,
    x_col="file_name",
    y_col=columns,
    batch_size=BATCH_SIZE,
    seed=None,
    shuffle=False,
    class_mode="raw", 
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    data_format='channels_last'
    )

test_generator = test_val_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=MAIN_PATH + IMAGES,
    x_col="file_name",
    batch_size=1,
    seed=None,
    shuffle=False,
    class_mode=None, # No targets are returned (the generator will only yield batches of image data, which is useful to use in model.predict()
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    data_format='channels_last'
    )


#assert type(test_val_datagen) == type(ImageDataGenerator())


Number of images in train set : 192
Number of images in val set : 48
Number of images in test set : 60


Found 192 validated image filenames.
Found 48 validated image filenames.
Found 60 validated image filenames.


In [15]:
'''----------------------------------
Create model (Functional API)
----------------------------------'''
tf.keras.backend.clear_session()

# Model tout pourri pour l'instant, transfert learning ensuite
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32)
x = Conv2D(16, 3, padding='same', activation='relu')(inputs)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Conv2D(32, 3, padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size = (2, 2))(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Dropout(rate=0.3)(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.4)(x)
x = Conv2D(128, 3, padding='same', activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Flatten()(x)
x = Dense(32)(x)

movement_output = Dense(NUM_MOVEMENT, activation = 'softmax', name='movement_output')(x) # First output for movement
genre_output = Dense(NUM_GENRE, activation = 'softmax', name='genre_output')(x) # Second outptu for genre

model = Model(inputs, [movement_output,genre_output]) # Create the model with 2 outputs


In [16]:
model.output_shape

[(None, 3), (None, 10)]

In [17]:
'''----------------------------------
Compile model
----------------------------------'''
model.compile(
    optimizer='adamax', 
    loss={'movement_output': CategoricalCrossentropy(), 
          'genre_output': CategoricalCrossentropy()},
    metrics={
        "movement_output": [
            Accuracy(),
            ],
        "genre_output": [
            Accuracy(),
            ]
        }
    )
# We have 2 losses one for each output

In [18]:
'''----------------------------------
Setup callbacks and tensorboard
----------------------------------'''

es = EarlyStopping(monitor='val_loss', patience=11, mode='min', restore_best_weights=True)
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=5, min_lr=1e-6)

%load_ext tensorboard
log_dir = f"logs/{USER}/{MODEL_NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tsboard = TensorBoard(log_dir=log_dir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


2021-08-25 18:07:39.573777: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-08-25 18:07:39.573798: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-08-25 18:07:39.574158: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


In [19]:
next(iter(train_generator))[0].shape

(32, 224, 224, 3)

In [20]:
next(iter(train_generator))[1][0].shape

(13,)

In [21]:
'''----------------------------------
Fit model
----------------------------------'''
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size # see steps_per_epoch comment below
STEP_SIZE_VAL = val_generator.n//val_generator.batch_size
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size



# We now have the model that has 2 output layers but our train_generator and valid_generator outputs a single array for the target labels, to handle this we need to write a python generator
# function that takes train_generator or valid_generator as input and yields a tuple containing the images and a list containing 2(No. of outputs) arrays for the target labels.
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
#         import ipdb; ipdb.set_trace()
        yield batch_x, [batch_y[:,:3], batch_y[:,3:]]




In [22]:
# The model does not see every image at each epoch !

history = model.fit(
    generator_wrapper(train_generator),
    steps_per_epoch=STEP_SIZE_TRAIN, # No 'batch_size' parameter for data coming from a generator or tf.Dataset, common choice is to use num_samples // batch_size
    epochs=EPOCHS,
    validation_data=generator_wrapper(val_generator), # We use a generator as the validation_data
    validation_steps=STEP_SIZE_VAL,
    callbacks=[es, rlrp, tsboard],
    verbose=1
    )

2021-08-25 18:07:51.199617: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
1/6 [====>.........................] - ETA: 14s - loss: 3.4013 - movement_output_loss: 1.1174 - genre_output_loss: 2.2838 - movement_output_accuracy: 0.0000e+00 - genre_output_accuracy: 0.0000e+00

2021-08-25 18:07:54.299854: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-08-25 18:07:54.299883: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


2/6 [=========>....................] - ETA: 12s - loss: 5.9993 - movement_output_loss: 1.9321 - genre_output_loss: 4.0672 - movement_output_accuracy: 0.0000e+00 - genre_output_accuracy: 0.0000e+00

2021-08-25 18:07:57.277063: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-25 18:07:57.280087: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-08-25 18:07:57.285016: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/gregoire/Custom v120210825-180739/train/plugins/profile/2021_08_25_18_07_57

2021-08-25 18:07:57.286603: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/gregoire/Custom v120210825-180739/train/plugins/profile/2021_08_25_18_07_57/MacBook-Pro-de-Camille-2.local.trace.json.gz
2021-08-25 18:07:57.297168: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/gregoire/Custom v120210825-180739/train/plugins/profile/2021_08_25_18_07_57

2021-08-25 18:07:57.297657: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.jso

6/6 [==============================] - 21s 4s/step - loss: 4.7853 - movement_output_loss: 1.6001 - genre_output_loss: 3.1852 - movement_output_accuracy: 0.0000e+00 - genre_output_accuracy: 0.0000e+00 - val_loss: 3.3824 - val_movement_output_loss: 1.1018 - val_genre_output_loss: 2.2806 - val_movement_output_accuracy: 0.0000e+00 - val_genre_output_accuracy: 0.0000e+00
Epoch 2/50
6/6 [==============================] - 18s 3s/step - loss: 3.3421 - movement_output_loss: 1.0831 - genre_output_loss: 2.2590 - movement_output_accuracy: 0.0000e+00 - genre_output_accuracy: 0.0000e+00 - val_loss: 3.3336 - val_movement_output_loss: 1.0951 - val_genre_output_loss: 2.2385 - val_movement_output_accuracy: 0.0000e+00 - val_genre_output_accuracy: 0.0000e+00
Epoch 3/50
6/6 [==============================] - 16s 3s/step - loss: 3.0908 - movement_output_loss: 1.0600 - genre_output_loss: 2.0308 - movement_output_accuracy: 0.0000e+00 - genre_output_accuracy: 0.0000e+00 - val_loss: 3.0895 - val_movement_output

KeyboardInterrupt: 

In [ ]:
'''----------------------------------
Evaluate
----------------------------------'''
test_generator.reset() # Need to reset the test_generator before whenever you call the predict_generator. This is important, if you forget to reset the test_generator you will get outputs in a weird order.
pred = model.evaluate(
    test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1
    )


In [ ]:
'''----------------------------------
Predict
----------------------------------'''
test_generator.reset() # Need to reset the test_generator before whenever you call the predict_generator. This is important, if you forget to reset the test_generator you will get outputs in a weird order.
pred = model.predict(
    test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1
    )


In [ ]:
pred